<a href="https://colab.research.google.com/github/giordamaug/BIONETdatasets/blob/main/TUD/notebooks/WLProb2Vec example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giordamaug/BIONETdatasets/main?filepath=TUD%2Fnotebooks%2FWLProb2Vec+example.ipynb)
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/giordamaug/BIONETdatasets/blob/main/TUD/notebooks/WLProb2Vec example.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# Cloning BIONETdatasets repository

In [ ]:
!git clone http://github.com/giordamaug/BIONETdatasets

# Install WLProb2Vec and python-igraph (if required)

In [1]:
# Install python-igraph (if required)
!pip install -q git+https://github.com/cds-group/WLProb2vec.git
!pip install -q python-igraph

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


# Load dataset from pickle dump

In [16]:
import pickle
sys.path.append('..')
from wrappers.igraph_wrapper import MyTUDataset
with open('BRAIN.obj', 'rb') as handle:
    dataset = pickle.load(handle) 
from collections import Counter
Counter(dataset.graph_labels)

Counter({0: 173, 1: 529})

# Load the dataset

In [14]:
dataname = 'MUTAG'
import sys, os
sys.path.append(os.path.abspath('BIONETdatasets/TUD'))
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../../methods'))
from wrappers.igraph_wrapper import MyTUDataset
import shutil
shutil.unpack_archive(f'../datasets/{dataname}.zip')
#shutil.unpack_archive(f'BIONETdatasets/TUD/datasets/{dataname}.zip')
dataset = MyTUDataset(dataname, path=f'{dataname}', verbose=True)
print(dataset.graph_list[0])

Loading edges:: 100%|██████████| 7442/7442 [00:00<00:00, 100247.65it/s]


Loading node labels...
Loading edge labels...
Loading graph labels...
IGRAPH UN-- 17 19 -- MUTAG_1
+ attr: class (g), name (g), label (v), name (v), label (e)
+ edges (vertex names):
0--1, 1--2, 2--3, 3--4, 4--5, 0--5, 4--6, 6--7, 7--8, 8--9, 3--9, 9--10,
10--11, 11--12, 12--13, 8--13, 12--14, 14--15, 14--16


# Create model and evaluate

In [12]:
import sys, os
import numpy as np
sys.path.append(os.path.abspath('BIONETdatasets/methods'))
if 'WLProb2Vec.wlprob2vec' in sys.modules:  
    del sys.modules["WLProb2Vec.wlprob2vec"]
if 'WLProb2Vec.WeisfeilerLehmanExt' in sys.modules:  
    del sys.modules["WLProb2Vec.WeisfeilerLehmanExt"]
from WLProb2Vec.wlprob2vec import wlprob2vec
#from netwld2v.WeisfeilerLehman import WeisfeilerLehman
#from netwld2v.graph2vec import Graph2Vec

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,matthews_corrcoef,accuracy_score,precision_score,f1_score, recall_score
from sklearn.model_selection import StratifiedKFold
from wrappers.igraph_wrapper import MyTUDataset
from time import time
# Set optimal params
params = {"annotation" : 'ndd', "wl_iterations" : 3, "dimensions": 64, "epochs": 100, "min_count": 2,  
  "seed": 1, "workers": 3, "vertex_label": 'label', "encodew": True}
# Transductive embedding
print("TRANSDUCTIVE EMBEDDING:")
#model = Graph2Vec(**params)
model = wlprob2vec(**params)
model.fit(dataset.graph_list)
X = model.get_embedding()
y = dataset.graph_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
clf = SVC(kernel='linear')
preds = clf.fit(X_train,y_train).predict(X_test)
print(confusion_matrix(y_test, preds),f'Acc. {accuracy_score(y_test, preds)}')
# Inductive embedding
print("INDUCTIVE EMBEDDING:")
G_train, G_test, y_train, y_test = train_test_split(dataset.graph_list, y, test_size=0.10, random_state=42)
model = wlprob2vec(**params)
model.fit(G_train)
X_train = model.get_embedding()
X_test = np.array(model.infer_vector(G_test))
clf = SVC(kernel='linear')
preds = clf.fit(X_train,y_train).predict(X_test)
print(confusion_matrix(y_test, preds),f'Acc. {accuracy_score(y_test, preds)}')

TRANSDUCTIVE EMBEDDING:
[[ 6  1]
 [ 0 12]] Acc. 0.9473684210526315
INDUCTIVE EMBEDDING:
[[ 5  2]
 [ 1 11]] Acc. 0.8421052631578947


In [17]:
import tqdm as tq
from time import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,matthews_corrcoef,accuracy_score,precision_score,f1_score, recall_score
from sklearn.model_selection import StratifiedKFold
from wrappers.igraph_wrapper import MyTUDataset
sys.path.append(os.path.abspath('BIONETdatasets/methods'))
from WLProb2Vec.wlprob2vec import wlprob2vec
params = {"annotation" : 'ndd', "wl_iterations" : 3, "dimensions": 128, "epochs": 100, "min_count": 2,  
   "seed": 1, "workers": 3, "vertex_label": None, "encodew": True}
start = time()
G = np.array(dataset.graph_list)
y = np.array(dataset.graph_labels)
cv_folds = 10 #@param {type:"slider", min:2, max:10, step:1}
tot_preds = np.array([])
tot_targets = np.array([])
tot_acc = np.array([])
tot_prec = np.array([])
tot_F1 = np.array([])
tot_recall = np.array([])
tot_MCC = np.array([])
model = wlprob2vec(**params)
X = model.fit(G).get_embedding()
skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
for train_index, test_index in tq.tqdm(list(skf.split(G,y)), desc="fold: "):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_pred = SVC(kernel='linear').fit(X_train,y_train).predict(X_test)
    tot_preds = np.append(tot_preds,y_pred)
    tot_targets = np.append(tot_targets,y_test)
    tot_acc = np.append(tot_acc, accuracy_score(y_test, y_pred))
    tot_prec = np.append(tot_prec, precision_score(y_test, y_pred, average='macro'))
    tot_F1 = np.append(tot_F1, f1_score(y_test, y_pred, average='macro'))
    tot_recall = np.append(tot_recall, recall_score(y_test, y_pred, average='macro'))
    tot_MCC = np.append(tot_MCC, matthews_corrcoef(y_test, y_pred))
temp = time() - start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
expired = '%d:%d:%d' %(hours,minutes,seconds)
print()
print(confusion_matrix(tot_targets, tot_preds))
print("Acc\t%.2f\u00B1%.2f"%((tot_acc * 100).mean(), (tot_acc * 100).std()))
print("Prec\t%.2f\u00B1%.2f"%(tot_prec.mean(), tot_prec.std()))
print("F1\t%.2f\u00B1%.2f"%(tot_F1.mean(), tot_F1.std()))
print("Recall\t%.2f\u00B1%.2f"%(tot_recall.mean(), tot_recall.std()))
print('MCC\t%.2f\u00B1%.2f'%(tot_MCC.mean(), tot_MCC.std()))
print('Elapsed time', expired)

fold: 100%|██████████| 10/10 [00:00<00:00, 18.06it/s]


[[143  30]
 [  0 529]]
Acc	95.72±1.43
Prec	0.97±0.01
F1	0.94±0.02
Recall	0.91±0.03
MCC	0.88±0.04
Elapsed time 0:49:52


In [14]:
import tqdm as tq
#params = {"annotation" : 'ndd', "wl_iterations" : 4, "dimensions": 128, "epochs": 100, "min_count": 2,  
#   "seed": 1, "workers": 3, "vertex_label": 'label', "encodew": True}
start = time()
G = np.array(dataset.graph_list)
y = np.array(dataset.graph_labels)
cv_folds = 10 #@param {type:"slider", min:2, max:10, step:1}
tot_preds = np.array([])
tot_targets = np.array([])
tot_acc = np.array([])
tot_prec = np.array([])
tot_F1 = np.array([])
tot_recall = np.array([])
tot_MCC = np.array([])
skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
for train_index, test_index in tq.tqdm(list(skf.split(G,y)), desc="fold: "):
    G_train, G_test = G[train_index], G[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = wlprob2vec(**params)
    X_train = model.fit(G_train).get_embedding()
    X_test = np.array(model.infer_vector(G_test))
    y_pred = SVC(kernel='linear').fit(X_train,y_train).predict(X_test)
    tot_preds = np.append(tot_preds,y_pred)
    tot_targets = np.append(tot_targets,y_test)
    tot_acc = np.append(tot_acc, accuracy_score(y_test, y_pred))
    tot_prec = np.append(tot_prec, precision_score(y_test, y_pred, average='macro'))
    tot_F1 = np.append(tot_F1, f1_score(y_test, y_pred, average='macro'))
    tot_recall = np.append(tot_recall, recall_score(y_test, y_pred, average='macro'))
    tot_MCC = np.append(tot_MCC, matthews_corrcoef(y_test, y_pred))
temp = time() - start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
expired = '%d:%d:%d' %(hours,minutes,seconds)
print()
print(confusion_matrix(tot_targets, tot_preds))
print("Acc\t%.2f\u00B1%.2f"%((tot_acc * 100).mean(), (tot_acc * 100).std()))
print("Prec\t%.2f\u00B1%.2f"%(tot_prec.mean(), tot_prec.std()))
print("F1\t%.2f\u00B1%.2f"%(tot_F1.mean(), tot_F1.std()))
print("Recall\t%.2f\u00B1%.2f"%(tot_recall.mean(), tot_recall.std()))
print('MCC\t%.2f\u00B1%.2f'%(tot_MCC.mean(), tot_MCC.std()))
print('Elapsed time', expired)

fold: 100%|██████████| 10/10 [00:15<00:00,  1.56s/it]


[[ 45  18]
 [ 21 104]]
Acc	79.24±7.27
Prec	0.78±0.08
F1	0.76±0.09
Recall	0.77±0.09
MCC	0.55±0.17
Elapsed time 0:0:15
